# hvPlot-Beispiele


## Installation

Um die Beispiele ausführen zu können, muss zusätzlich [Snappy](https://github.com/google/snappy) installiert werden.

Mit [Spack](https://jupyter-tutorial.readthedocs.io/de/latest/productive/envs/spack/index.html) könnt ihr Snappy in eurem Kernel bereitstellen, z.B. mit:

```
$ spack env activate python-374
$ spack install snappy
```

Alternativ könnt ihr Snappy auch mit anderen Paketmanagern installieren, z.B.

* Für Debian/Ubuntu:

  ```
  $ sudo apt install libsnappy-dev
  ```

* Für Mac OS:

  ```
  $ brew install snappy
  ```

Anschließend sollten für euren Kernel noch weitere Pakete installiert werden, z.B. mit:

  ```
  $ pipenv install intake intake-parquet s3fs python-snappy pyviz-comms
  …
  ```


## Einführung

Als erstes importieren wir Numpy und Pandas um anschließend einen kleinen Satz zufälliger Daten zu erstellen:

In [ ]:
import numpy as np
import pandas as pd

index = pd.date_range('1/1/2000', periods=1000)
df = pd.DataFrame(np.random.randn(1000, 4), index=index, columns=list('ABCD')).cumsum()

df.head()

### Pandas`.plot ()`-API

Pandas bietet standardmäßig Matplotlib-basiertes Plotten mit der `.plot()`-Methode:

In [ ]:
%matplotlib inline

df.plot();

Das Ergebnis ist ein PNG-Bild, das leicht angezeigt werden kann, ansonsten aber statisch ist.

> **Hinweis:** In Pandas > 0.25.0 kann das Backend ausgetauscht werden, z.B. mit `pd.options.backend.plotting == 'holoviews',`. Weitere Informationen hierzu findet ihr unter [Pandas-API](https://hvplot.holoviz.org/user_guide/Pandas_API.html).

### `.hvplot()`

Wenn wir statt `%matplotlib inline` zu `import hvplot.pandas` und der `df.hvplot`-Methode wechseln, , wird jetzt ein interaktiv erforschbares Bokeh-Diagramm erzeugt mit Verschieben und Vergrößern/Verkleinern sowie anklickbaren Legenden:

In [ ]:
import hvplot.pandas

df.hvplot()

Ein solches interaktive Diagramm erleichtert das Erkunden der der Daten erheblich, ohne dass hierfür zusätzlicher Code geschrieben werden müsste.

### Native `hvPlot`-API 

Für das obige Diagramm hat hvPlot die Pandas-`.hvplot()`-Methode dynamisch hinzugefügt, sodass ihr dieselbe Syntax wie bei Pandas-Plots verwenden könnt. Wenn ihr ein expliziteres Vorgehen bevorzugt,könnt ihr stattdessen direkt mit den hvPlot-Objekten arbeiten:

In [ ]:
from hvplot import hvPlot
import holoviews as hv
hv.extension('bokeh')

plot = hvPlot(df)
plot(y=['A', 'B', 'C', 'D'])

### Hilfe

Wenn ihr in IPython oder Jupyter-Notebooks arbeitet, vervollständigen die `hvplot`-Methoden automatisch gültige Schlüsselwörter. Wenn ihr beispielsweise nach dem Deklarieren des Plottyps die Tabulatortaste drücken, werden alle gültigen Schlüsselwörter und die Dokumentzeichenfolge angezeigt:

```
df.hvplot.line(<TAB>
```

Außerhalb einer interaktiven Umgebung werden mit `hvplot.help` alle Informationen für einen Diagrammtyp angezeigt, z.B.:

In [ ]:
hvplot.help('line')

> **Siehe auch:** Weitere Informationen zu den verfügbaren Optionen erhaltet ihr in [Customization](https://hvplot.holoviz.org/user_guide/Customization.html).

## Plotting

In den folgenden Beispielen wird neben der pandas- auch die dask-hvPlot-API verwendet:

In [ ]:
import hvplot.dask

Das `hvplot.sample_data`-Modul erstellt diese Datensätze als Intake-Datenkataloge, die wir mit Pandas laden können:

In [ ]:
from hvplot.sample_data import us_crime, airline_flights

crime = us_crime.read()
print(type(crime))
crime.head()

Alternativ können wir `dask.DataFrame` verwenden:

In [ ]:
flights = airline_flights.to_dask().persist()
print(type(flights))
flights.head()

### Die Plot-API

Die Schinttstellen

* `dask.dataframe.DataFrame.hvplot`
* `pandas.DataFrame.hvplot`
* `intake.DataSource.plot`

und ihre `Series`-Äquivalente bieten eine leistungsfähige High-Level-API um auch komplexe Plots erzeugen zu können. Dabei kann die `.hvplot`-API entweder direkt oder als Namespace verwendet werden, um bestimmte Plottypen zu generieren.

Die expliziteste Methode zur Verwendung der Plot-API besteht darin, die Namen der Spalten anzugeben, die auf der x- bzw. y-Achse geplottet werden sollen:

In [ ]:
crime.hvplot.line(x='Year', y='Violent Crime rate')

Zusaätzlich kann auch noch der Diagrammtyp mit `kind` angegeben werden:

In [ ]:
crime.hvplot(x='Year', y='Violent Crime rate', kind='scatter')

Mit der `by`-Variable könnt ihr die Daten in einer oder mehreren zusätzlichen Spalten gruppieren. Als Beispiel wird im Folgenden die Abfahrtsverzögerung (`'depdelay'`) als Funktion der `'distance'` dargestellt und die Daten nach `'carrier'` gruppiert:

In [ ]:
flight_subset = flights[flights.carrier.isin([b'OH', b'F9'])]
flight_subset.hvplot(x='distance', y='depdelay', by='carrier', kind='scatter', alpha=0.2, persist=True)

Im obigen Beispiel haben wir die x- und y-Achsen explizit angegeben.

Andernfalls würde für die x-Achse die Pandas-Indexspalte verwendet werden und für die y-Achse alle Nicht-Index-Spalten mit der Standardbezeichnung `'value'`. Wollt ihr nur die y-Achsenbezeichnung explizit angeben, so steht euch die `value_label`-Option zur Verfügung.

In [ ]:
crime.hvplot(x='Year', y=['Violent Crime rate', 'Robbery rate', 'Burglary rate'],
             value_label='Rate (per 100k people)')

### Der `hvplot`-Namespace 

Statt des k`ind`-Argument können wir auch den `hvplot`-Namespace für den Plotaufruf zu verwenden. Die unterstützten Plottypen lassen sich leicht ermitteln mit der Tab-Vervollständigung, also
```
crime.hvplot.<TAB>

```

Verfügbare Diagrammtypen sind:

* [area()](#area()) zeichnet ein Flächendiagramm ähnlich einem Liniendiagramm, außer dass der Bereich unter der Kurve gefüllt und optional gestapelt wird
* [bar()](#bar())zeichnet ein Flächendiagramm ähnlich einem Liniendiagramm, außer dass der Bereich unter der Kurve gefüllt und optional gestapelt wird
* [bivariate()](#bivariate()) zeichnet ein Flächendiagramm ähnlich einem Liniendiagramm, außer dass der Bereich unter der Kurve gefüllt und optional gestapelt wird
* [box()](#box()) zeichnet ein [Box-Whisker-Diagramm](https://de.wikipedia.org/wiki/Box-Plot), in dem die Verteilung einer oder mehrerer Variablen verglichen wird
* [heatmap()](#heatma()p) zeichnet Hex-Bins
* [hexbins()](#hexbins()) zeichnet die Verteilung eines oder mehrerer Histogramme als Satz von Containern
* [histogram()](#histogram()) zeichnet die Kernel-Dichteschätzung einer oder mehrerer Variablen
* [kde()](#kde(),-density()) zeichnet die Kernel-Dichteschätzung einer oder mehrerer Variablen
* [line()](#line()) zeichnet ein Liniendiagramm (z. B. für eine Zeitreihe)
* [step()](#step()) zeichnet ein Schrittdiagramm, das einem Liniendiagramm ähnelt
* [scatter()](#scatter()) zeichnet ein Streudiagramm, in dem zwei Variablen verglichen werden 
* [table()](#table()) erzeugt eine SlickGrid-Datentabelle
* [violin()](#violin()) zeichnet ein Violinen-Diagramm, in dem die Verteilung einer oder mehrerer Variablen mithilfe der Kernel-Dichteschätzung verglichen wird

### `area()`

Wie die meisten anderen Diagrammtypen unterstützt das `area`-Diagramm die drei oben beschriebenen Möglichkeiten zum Definieren eines Diagramms. Ein Flächendiagramm ist am nützlichsten, wenn mehrere Variablen in einem gestapelten Diagramm dargestellt werden. Dies kann durch Angabe für die `x`-, `y`- und `by`-Spalten erreicht werden oder mit `columns` und `index`/`use_index` als Optionen für die `x`-Achse.

In [ ]:
crime.hvplot.area(x='Year', y=['Robbery', 'Aggravated assault'])

Wir können auch explizit `stacked` auf `False` setzen und einen `alpha`-Wert definieren und um die Werte direkt vergleichen zu können:

In [ ]:
crime.hvplot.area(x='Year', y=['Aggravated assault', 'Robbery'], stacked=False, alpha=0.4)

Eine andere Verwendung für ein Flächendiagramm besteht darin, die Streuung eines Werts zu visualisieren. Wenn wir beispielsweise den Flugdatensatz verwenden, möchten wir möglicherweise die Streuung der mittleren Verspätungswerte zwischen den Fluggesellschaften sehen. Zu diesem Zweck berechnen wir die mittlere Verzögerung nach Tag und Carrier und dann die minimale/maximale mittlere Verzögerung für alle Carrier. Da die Ausgabe von `hvplot` nur ein reguläres Holoviews-Objekt ist, können wir den Overlay-Operator (`*`) verwenden, um die Diagramme übereinander zu platzieren.

In [ ]:
delay_min_max = flights.groupby(['day', 'carrier'])['carrier_delay'].mean().groupby('day').agg([np.min, np.max])
delay_mean = flights.groupby('day')['carrier_delay'].mean()

delay_min_max.hvplot.area(x='day', y='amin', y2='amax', alpha=0.2) * delay_mean.hvplot()

### `bar()`

Im einfachsten Fall können wir `.hvplot.bar` verwenden. Um die Beschriftung auf der x-Achse um 90° zu drehen, geben wir noch `rot=90` an.

In [ ]:
crime.hvplot.bar(x='Year', y='Violent Crime rate', rot=90)

Wenn wir stattdessen mehrere Spalten vergleichen möchten, können wir eine Liste von Spalten festlegen. Mit der `stacked`-Option können wir dann die Spaltenwerte einfacher vergleichen:

In [ ]:
crime.hvplot.bar(x='Year', y=['Violent crime total', 'Property crime total'],
                 stacked=True, rot=90, width=800, legend='top_left')

### `scatter()`

Das Streudiagramm unterstützt viele der Funktionen der obigen Diagrammtypen, kann jedoch mit der `c`-Option auch  eingefärbt werden.

In [ ]:
crime.hvplot.scatter(x='Violent Crime rate', y='Burglary rate', c='Year')

Um Farbe zur Darstellung einer Dimension zu verwenden, kann die `cmap`-Option genutzt werden werden, um die zu verwendende Farbkarte anzugeben. Zusätzlich kann die Farbleiste deaktiviert werden mit `colorbar=False`.

### `step()`

Ein Schrittdiagramm ist einem Liniendiagramm sehr ähnlich, aber anstatt linear zwischen Abtastwerten zu interpolieren, visualisiert das Schrittdiagramm diskrete Schritte. Die Position der Schritte kann mit dem `where`-Schlüsselwort un den Werten `'pre'`, `'mid'` (Standard) und `'post'` gesteuert werden.

In [ ]:
crime.hvplot.step(x='Year', y=['Robbery', 'Aggravated assault'])

### `hexbin()`

Mit der `hexbin`-Methode können Sie hexagonale Bin-Diagramme erstellen. Sie können eine nützliche Alternative zu Streudiagrammen sein, wenn die Daten zu dicht sind, um jeden Punkt einzeln zu zeichnen. Da unsere Flugdaten nicht gleichmäßig auf einer linearen Skala verteilt sind, verwenden wir die `logz`-Option für eine logarithmische Skala.

In [ ]:
flights.hvplot.hexbin(x='airtime', y='arrdelay', width=600, height=500, logz=True)

### `bivariate()`

Mit der `bivariate`-Methode könnt ihr ein 2D-Dichtediagramm erstellen. Bivariate Diagramme sind neben Hexbin-Diagrammen eine weitere Alternative zu Streudiagrammen, wenn die Daten zu dicht sind, um jeden Punkt einzeln zu zeichnen.

In [ ]:
crime.hvplot.bivariate(x='Violent Crime rate', y='Burglary rate', width=600, height=500)

### `heatmap()`

`HeatMap` kann die Beziehung zwischen drei Variablen anzeigen und neben den Variablen `'x'` und `'y'` zusätzlich `'C'` anzeigen. Zusätzlich werden mit der `reduce_function` die Werte für jeden Container aus den Stichproben berechnet.

In [ ]:
flights.compute().hvplot.heatmap(x='day', y='carrier', C='depdelay', reduce_function=np.mean, colorbar=True)

### `table()`

Im Gegensatz zu allen anderen Plottypen kann für eine Tabelle nur angegeben werden, ob alle Spalten oder mit `columns` nur eine Teilmenge angezeigt werden soll.

In [ ]:
crime.hvplot.table(columns=['Year', 'Population', 'Violent Crime rate'], width=400)

### `hist()`

Das Zeichnen von Verteilungen unterscheidet sich geringfügig von anderen Plots, da sie im einfachen Fall nur eine Variable darstellen. Daher muss bei diesem Plottyp kein `index` oder `x`-Wert angegeben werden, sondern

* deklariert eine einzelne `y`-Variable, z.B. `source.plot.hist(variable)` oder
* deklariert eine `y`-Variable und eine `by`-Variable, z.B. `source.plot.hist(variable, by='Group')` oder
* deklariert Spalten oder zeichnet alle Spalten, z.B. `source.plot.hist()` oder `source.plot.hist(columns=['A', 'B', 'C'])`

In [ ]:
crime.hvplot.hist(y='Violent Crime rate')

Alternativ önnen wir auch die Verteilung mehrerer Spalten darstellen:

In [ ]:
columns = ['Violent Crime rate', 'Property crime rate', 'Burglary rate']
crime.hvplot.hist(y=columns, bins=50, alpha=0.5, legend='top', height=400)

Wir können die Daten auch nach anderen Variablen gruppieren und die Carrier in eigene `subplots` aufteilen:

In [ ]:
flight_subset = flights[flights.carrier.isin([b'AA', b'US', b'OH'])]
flight_subset.hvplot.hist('depdelay', by='carrier', bins=20, bin_range=(-20, 100), width=300, subplots=True)

### `kde()`, `density()`

Ihr könnt Dichtediagramme auch mit `hvplot.kde()` oder `hvplot.density()` erstellen:

In [ ]:
crime.hvplot.kde(y='Violent Crime rate')

Der Vergleich der Verteilung mehrerer Spalten ist ebenfalls möglich:

In [ ]:
columns=['Violent Crime rate', 'Property crime rate', 'Burglary rate']
crime.hvplot.kde(y=columns, alpha=0.5, value_label='Rate', legend='top_right')

`hvplot.kde` unterstützt auch das `by`-Schlüsselwort:

In [ ]:
flight_subset = flights[flights.carrier.isin([b'AA', b'US', b'OH'])]
flight_subset.hvplot.kde('depdelay', by='carrier', xlim=(-20, 70), width=300, subplots=True)

### `box()`

Genau wie die anderen verteilungsbasierten Diagrammtypen unterstützt das  [Box-Whisker-Diagramm](https://de.wikipedia.org/wiki/Box-Plot) das Zeichnen einer einzelnen Spalte:

In [ ]:
crime.hvplot.box(y='Violent Crime rate')

Es unterstützt auch mehrere Spalten und die gleichen Optionen wie berits oben genannt:`legend`, `invert` und `value_label`:

In [ ]:
columns=['Burglary rate', 'Larceny-theft rate', 'Motor vehicle theft rate',
         'Property crime rate', 'Violent Crime rate']
crime.hvplot.box(y=columns, group_label='Crime', legend=False, value_label='Rate (per 100k)', invert=True)

Auch die Verwendung des `by`-Schlüsselworts zum Aufteilen der Daten in mehrere Teilmengen wird unterstützt:

In [ ]:
flight_subset = flights[flights.carrier.isin([b'AA', b'US', b'OH'])]
flight_subset.hvplot.box('depdelay', by='carrier', ylim=(-10, 70))

### Zusammengesetzte Diagramme

Eine der Hauptstärken von HoloViews ist die einfache Erstellung verschiedener Diagramme. Einzelne Diagramme können mit den Operatoren `*` und `+` überlagert bzw. zusammengesetzt werden.

> **Siehe auch:** [Composing Elements](http://holoviews.org/user_guide/Composing_Elements.html)

In [ ]:
crime.hvplot(x='Year', y='Violent Crime rate') * crime.hvplot.scatter(x='Year', y='Violent Crime rate', c='k')

Wir können auch verschiedene Diagramme und Tabellen zusammen erstellen:

In [ ]:
(crime.hvplot.bar(x='Year', y='Violent Crime rate', rot=90, width=550) +
 crime.hvplot.table(['Year', 'Population', 'Violent Crime rate'], width=420))

### Big Data

In den vorherigen Beispielen fassten wir den relativ große Airline-Datensatz zusammen indem wir Teilmengen für die Darstellung bildeten. Stattdessen können wir die Daten jedoch auch mithilfe von [Datashader](https://jupyter-tutorial.readthedocs.io/de/latest/viz/bokeh/integration/datashader.html) aggregieren, wobei der gesamte verfügbare Rohdatensatz gerendert wird (sofern die Auflösung des Bildschirms dies zulässt).

In [ ]:
flights.hvplot.scatter(x='distance', y='airtime', datashade=True)

### `groupby`

Dank der Fähigkeit von HoloViews, einen Parameterraum mit einer Reihe von Widgets zu erkunden, können wir eine Gruppe entlang einer bestimmten Spalte oder Dimension anwenden, z.B. die Verteilung der Abflugverzögerungen nach Carriern und Tag gruppiert anzeigen, wobei Benutzer\*innen auswählen können, welcher Tag angezeigt werden soll:

In [ ]:
flights.hvplot.violin(y='depdelay', by='carrier', groupby='dayofweek', ylim=(-20, 60), height=500)